In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor

In [2]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

base = base[base.price > 10]
base = base.loc[base.price < 350000]

valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
 
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [4]:
def criar_rede():
    
    regressor = Sequential()
    
    regressor.add(Dense(units = 158, activation = 'relu', input_dim = 316))
    
    regressor.add(Dense(units = 158, activation = 'relu'))
    
    regressor.add(Dense(units = 1, activation = 'linear'))
    
    regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                      metrics = ['mean_absolute_error'])
    
    return regressor

In [5]:
regressor = KerasRegressor(build_fn = criar_rede,
                           epochs = 100,
                           batch_size = 300)

In [10]:
resultados = cross_val_score(estimator = regressor,
                             X = previsores, y = preco_real,
                             cv = 10, scoring = 'neg_mean_absolute_error')

Epoch 1/100
1078/1078 [==============================] - 3s 3ms/step - loss: 3886.6541 - mean_absolute_error: 3886.6541
Epoch 2/100
1078/1078 [==============================] - 3s 3ms/step - loss: 3455.4539 - mean_absolute_error: 3455.4539
Epoch 3/100
1078/1078 [==============================] - 3s 3ms/step - loss: 3237.4192 - mean_absolute_error: 3237.4192
Epoch 4/100
1078/1078 [==============================] - 3s 3ms/step - loss: 3004.6829 - mean_absolute_error: 3004.6829
Epoch 5/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2870.3452 - mean_absolute_error: 2870.3452
Epoch 6/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2806.4751 - mean_absolute_error: 2806.4753
Epoch 7/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2750.3313 - mean_absolute_error: 2750.3313
Epoch 8/100
1078/1078 [==============================] - 3s 3ms/step - loss: 2703.8762 - mean_absolute_error: 2703.8762
Epoch 9/100
1078/1078 [=================

In [13]:
resultados

array([-2181.31390681, -2289.01789486, -2209.5126328 , -2241.29931554,
       -2167.28117746, -2224.15188233, -2205.33448496, -2240.80353299,
       -2162.27389535, -2183.98987699])

In [11]:
media = resultados.mean()
media

-2210.4978600082795

In [12]:
desvio = resultados.std()
desvio

37.408864586406864